## Dataset Uploading


In [1]:
!pip install kaggle

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 2.1 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.4 MB 2.1 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.4 MB 2.1 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.4 MB 882.6 kB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.4 MB 882.6 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 662.0 kB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 662.0 kB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 697.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   -------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.5.1 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 9.6.0 which is incompatible.


In [ ]:
import os
import shutil

kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_dir, exist_ok=True)


shutil.copy("kaggle.json", kaggle_dir)


os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 600)

print("Kaggle API token configured successfully!")


In [9]:
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other




  0%|          | 0.00/80.9M [00:00<?, ?B/s]
  1%|          | 1.00M/80.9M [00:01<02:06, 664kB/s]
  2%|▏         | 2.00M/80.9M [00:01<01:02, 1.32MB/s]
  4%|▎         | 3.00M/80.9M [00:01<00:39, 2.08MB/s]
  6%|▌         | 5.00M/80.9M [00:02<00:19, 4.06MB/s]
  9%|▊         | 7.00M/80.9M [00:02<00:13, 5.95MB/s]
 11%|█         | 9.00M/80.9M [00:02<00:09, 8.24MB/s]
 14%|█▎        | 11.0M/80.9M [00:02<00:07, 10.2MB/s]
 16%|█▌        | 13.0M/80.9M [00:02<00:06, 11.5MB/s]
 19%|█▊        | 15.0M/80.9M [00:02<00:05, 13.1MB/s]
 21%|██        | 17.0M/80.9M [00:02<00:04, 13.8MB/s]
 23%|██▎       | 19.0M/80.9M [00:02<00:04, 15.3MB/s]
 26%|██▌       | 21.0M/80.9M [00:03<00:04, 14.6MB/s]
 28%|██▊       | 23.0M/80.9M [00:03<00:03, 15.7MB/s]
 31%|███       | 25.0M/80.9M [00:03<00:03, 16.2MB/s]
 33%|███▎      | 27.0M/80.9M [00:03<00:03, 16.0MB/s]
 36%|███▌      | 29.0M/80.9M [00:03<00:03, 16.6MB/s]
 38%|███▊      | 31.0M/80.9M [00:03<00:03, 16.1MB/s]
 41%|████      | 33.0M/80.9M [00:03<00:03, 16.7MB/s]
 4

In [2]:
from zipfile import ZipFile

zip_path = "sentiment140.zip"   

with ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("sentiment140")  

print("Dataset extracted successfully!")



Dataset extracted successfully!


## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tauhid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
## Read dataset
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
tweets_df = pd.read_csv("sentiment140/training.1600000.processed.noemoticon.csv",names=column_names, encoding='ISO-8859-1',header=None)

In [7]:
tweets_df.shape

(1600000, 6)

In [8]:
tweets_df.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
tweets_df.isnull().sum()

## No missing values in the dataset

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
tweets_df['target'].value_counts()


# 0 - Negative Sentiment
# 4 - Positive Sentiment

target
0    800000
4    800000
Name: count, dtype: int64

In [9]:
tweets_df.replace({'target': {4:1}}, inplace=True)
## Converted 4 to 1 for positive sentiment

In [11]:
tweets_df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [10]:
## Stemming
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content) 
    return stemmed_content

In [14]:
tweets_df['stemmed_text'] = tweets_df['text'].apply(stemming)

In [15]:
tweets_df.head(5)

,target,ids,date,flag,user,text,stemmed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [16]:
print(tweets_df['stemmed_text'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_text, Length: 1600000, dtype: object


In [17]:
## Separating features and target
X = tweets_df['stemmed_text'].values
y = tweets_df['target'].values

In [18]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [19]:
print(y)

[0 0 0 ... 1 1 1]


In [20]:
## Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=2)

In [21]:
print(X_train.shape,X_test.shape)

(1280000,) (320000,)


In [22]:
## COnvert Textual column into numerical using TF-IDF Vectorization
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [23]:
print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [26]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

## Model Training

In [24]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": LinearSVC(),
    "Naive Bayes": MultinomialNB()
}


for i in range (len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    #make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    ##Training Accuracy
    model_train_accuracy = accuracy_score(y_train, y_train_pred)
    
    ##Testing Accuracy
    model_test_accuracy = accuracy_score(y_test, y_test_pred)


    print(list(models.keys())[i])

    
    print("Model Performance for Training Set")
    print("-Accuracy: {:.4f}".format(model_train_accuracy))
    print('-------------------------------------------------------')
    print("Model Performance for Testing Set")
    print("-Accuracy: {:.4f}".format(model_test_accuracy))


Logistic Regression
Model Performance for Training Set
-Accuracy: 0.7987
-------------------------------------------------------
Model Performance for Testing Set
-Accuracy: 0.7767
Linear SVM
Model Performance for Training Set
-Accuracy: 0.8623
-------------------------------------------------------
Model Performance for Testing Set
-Accuracy: 0.7697
Naive Bayes
Model Performance for Training Set
-Accuracy: 0.8200
-------------------------------------------------------
Model Performance for Testing Set
-Accuracy: 0.7558


## HyperParameter Tuning Logistic Regression 

In [32]:
params = {
    "C": [0.01,10],
    "solver": ["liblinear", "lbfgs"],
    "max_iter": [500, 1000]
}

In [33]:
params

{'C': [0.01, 10], 'solver': ['liblinear', 'lbfgs'], 'max_iter': [500, 1000]}

In [34]:
## Using RandomSearch CV
randomcv_models = [
                    ("LR", LogisticRegression(), params),
]

In [35]:
randomcv_models

[('LR',
  LogisticRegression(),
  {'C': [0.01, 10],
   'solver': ['liblinear', 'lbfgs'],
   'max_iter': [500, 1000]})]

In [36]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=10,
                                cv=3,
                                verbose=2,
                                n_jobs=-1
                                )

In [ ]:
random.fit(X_train,y_train)
model_param[name] = random.best_params_

for model_name in model_param:
    print(f"----------------Best Param for {model_name}---------------")
    print(model_param[model_name])

In [41]:
models = {
    "Logistic Regression" : LogisticRegression(  
    solver='liblinear',
    max_iter=1000)
}

In [42]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    ##Training Accuracy
    model_train_accuracy = accuracy_score(y_train, y_train_pred)
    
    ##Testing Accuracy
    model_test_accuracy = accuracy_score(y_test, y_test_pred)


    print(list(models.keys())[i])

    
    print("Model Performance for Training Set")
    print("-Accuracy: {:.4f}".format(model_train_accuracy))
    print('-------------------------------------------------------')
    print("Model Performance for Testing Set")
    print("-Accuracy: {:.4f}".format(model_test_accuracy))


Logistic Regression
Model Performance for Training Set
-Accuracy: 0.8102
-------------------------------------------------------
Model Performance for Testing Set
-Accuracy: 0.7780


In [43]:
import pickle
filename = "best_model.pkl"
pickle.dump(model, open(filename,'wb'))
pickle.dump(tfidf, open("vectorizer.pkl", "wb"))